<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset el primer libro de Harry Potter.

In [28]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('Book 1-The Philosopher\'s Stone.txt', sep='/n', header=None)
df.head()

/tmp/ipykernel_8597/665039940.py:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,THE BOY WHO LIVED
1,"Mr. and Mrs. Dursley, of number four, Privet D..."
2,were proud to say that they were perfectly nor...
3,thank you very much. They were the last people...
4,expect to be involved in anything strange or


In [29]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 10143


### 1 - Preprocesamiento

In [30]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [31]:
# Demos un vistazo
sentence_tokens[:2]

[['the', 'boy', 'who', 'lived'],
 ['mr', 'and', 'mrs', 'dursley', 'of', 'number', 'four', 'privet', 'drive']]

### 2 - Crear los vectores (word2vec)

In [32]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [33]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [34]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [35]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 10143


In [36]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1794


### 3 - Entrenar el modelo generador

In [37]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=200,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 685916.5625
Loss after epoch 1: 483231.1875
Loss after epoch 2: 435262.25
Loss after epoch 3: 426776.25
Loss after epoch 4: 379181.75
Loss after epoch 5: 367411.25
Loss after epoch 6: 364376.75
Loss after epoch 7: 360744.25
Loss after epoch 8: 357541.0
Loss after epoch 9: 353716.25
Loss after epoch 10: 327654.5
Loss after epoch 11: 326120.0
Loss after epoch 12: 322325.5
Loss after epoch 13: 318897.0
Loss after epoch 14: 316822.5
Loss after epoch 15: 312564.5
Loss after epoch 16: 308766.0
Loss after epoch 17: 306620.5
Loss after epoch 18: 304913.5
Loss after epoch 19: 301543.0
Loss after epoch 20: 299611.5
Loss after epoch 21: 297775.5
Loss after epoch 22: 296261.5
Loss after epoch 23: 285513.0
Loss after epoch 24: 274081.0
Loss after epoch 25: 272453.0
Loss after epoch 26: 270017.0
Loss after epoch 27: 268892.0
Loss after epoch 28: 268538.0
Loss after epoch 29: 266518.0
Loss after epoch 30: 265656.0
Loss after epoch 31: 265299.0
Loss after epoch 32: 263628.0
Loss af

(11023381, 17013200)

### 4 - Ensayar

In [38]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["magic"], topn=10)

[('proper', 0.3462655544281006),
 ('ordinary', 0.3452013432979584),
 ('d', 0.3074802756309509),
 ('goblins', 0.30632904171943665),
 ('often', 0.3060491383075714),
 ('history', 0.30495935678482056),
 ('case', 0.30302831530570984),
 ('ministry', 0.2988910973072052),
 ('breaking', 0.2946619391441345),
 ('today', 0.2908533811569214)]

In [39]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["hermione"], topn=10)

[('hooch', 0.32403385639190674),
 ('“get', 0.31105950474739075),
 ('herself', 0.29633060097694397),
 ('leaning', 0.2941463589668274),
 ('fallen', 0.2879769504070282),
 ('knight', 0.2871142327785492),
 ('sharply', 0.2866591215133667),
 ('attention', 0.28014370799064636),
 ('stranger', 0.27895069122314453),
 ('hate', 0.2783515155315399)]

In [40]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["ron"], topn=10)

[('mumbled', 0.2949409484863281),
 ('sounding', 0.29155096411705017),
 ('shrieked', 0.289141982793808),
 ('stretched', 0.2875233292579651),
 ('moaned', 0.2866344749927521),
 ('hissed', 0.2818051874637604),
 ('anxiously', 0.2722613215446472),
 ('hermione', 0.271834135055542),
 ('sheets', 0.26614806056022644),
 ('fine', 0.2646057903766632)]

In [41]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["dursley"], topn=10)

[('mrs', 0.3985974192619324),
 ('mr', 0.3735896348953247),
 ('drove', 0.36803898215293884),
 ('draco', 0.3406873643398285),
 ('crying', 0.33062708377838135),
 ('figg', 0.3294597268104553),
 ('reading', 0.3238988518714905),
 ('nicolas', 0.3194156885147095),
 ('lily', 0.3174111545085907),
 ('struck', 0.31458166241645813)]

In [42]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["malfoy"], topn=10)

[('draco', 0.396837055683136),
 ('goyle', 0.3384241759777069),
 ('mine', 0.3341669738292694),
 ('grinned', 0.32287296652793884),
 ('crabbe', 0.31418469548225403),
 ('frightened', 0.28948330879211426),
 ('packed', 0.28671708703041077),
 ('thousand', 0.2860214114189148),
 ('laughter', 0.2789485454559326),
 ('knees', 0.2785743474960327)]

In [43]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["hogwarts"], topn=10)

[('ground', 0.06237461790442467),
 ('feeling', 0.042962513864040375),
 ('lay', 0.042097222059965134),
 ('sat', 0.040914855897426605),
 ('legs', 0.03921947628259659),
 ('forgotten', 0.0310788806527853),
 ('water', 0.025649912655353546),
 ('also', 0.01710178516805172),
 ('sound', 0.01603684015572071),
 ('filch', 0.014373709447681904)]

In [44]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["muggle"], topn=10)

[('dressed', 0.3774559199810028),
 ('families', 0.35730692744255066),
 ('“not', 0.350401371717453),
 ('beans', 0.34601160883903503),
 ('er', 0.33243003487586975),
 ('nothin’', 0.33231690526008606),
 ('“yeh', 0.33118414878845215),
 ('meeting', 0.32793739438056946),
 ('our', 0.32603248953819275),
 ('gets', 0.3247361481189728)]

In [45]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["gryffindor"], topn=10)

[('tower', 0.34224411845207214),
 ('gryffindors', 0.33077874779701233),
 ('goal', 0.323725163936615),
 ('points', 0.3173125386238098),
 ('afford', 0.31698521971702576),
 ('posts', 0.3132522702217102),
 ('thrown', 0.3132173418998718),
 ('gotta', 0.3129391372203827),
 ('everywhere', 0.3079492151737213),
 ('marcus', 0.3055194914340973)]

In [46]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["hagrid"], topn=5)

[('chest', 0.33109748363494873),
 ('jus’', 0.3181031048297882),
 ('anxiously', 0.3158767521381378),
 ('umbrella', 0.30286911129951477),
 ('nearest', 0.29883819818496704)]

In [47]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: "Key 'diedaa' not present in vocabulary"

### 5 - Visualizar agrupación de vectores

In [48]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [49]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show()

# Conclusiones

Al ser mucho más grande el dataset el entrenamiento es más costoso que analizar las letras de canciones de una banda. Por el mismo motivo al existir muchas más palabras, las relaciones entre las mismas se vuelven más difusas.

De todas formas el desempeño en general es "bueno", las palabras más relacionadas con cada token utilizado tienen cierto sentido.

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.